# Trend Screener

In [93]:
pip install arch scipy yfinance pandas numpy requests QuantLib-Python


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [94]:
import datetime as dt
import pandas as pd
import yfinance as yf
import numpy as np
from arch import arch_model
from scipy.stats import norm
import QuantLib as ql
from datetime import datetime
from multiprocessing import Pool, cpu_count
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests

In [95]:
tickers = ['CLOV', 'ASTS', 'WGS', 'NSSC', 'AAPL', 'IESC', 'HRMY', 'WLFC', 'FTAI']

risk_free_rate = 0.05

end_date = '2024-09-27'

start_date_1d = (pd.to_datetime(end_date) - pd.DateOffset(days=1)).strftime('%Y-%m-%d')
start_date_3m = (pd.to_datetime(end_date) - pd.DateOffset(months=3)).strftime('%Y-%m-%d')
start_date_12m = (pd.to_datetime(end_date) - pd.DateOffset(months=12)).strftime('%Y-%m-%d')


stock_prices = yf.download(tickers, start = start_date_1d, end = end_date)

[*********************100%%**********************]  9 of 9 completed


In [96]:
def convert_expiry_date(expiry_str):
    month_map = {
        "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4,
        "May": 5, "Jun": 6, "Jul": 7, "Aug": 8,
        "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12
    }
    
    month_str, day = expiry_str.split()
    day = int(day)

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month

    month = month_map[month_str]

    if month < current_month:
        year = current_year + 1
    else:
        year = current_year

    formatted_date = f"{year:04d}-{month:02d}-{day:02d}"
    
    return formatted_date

def convert_to_float(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return None
    
def convert_to_int(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return None


def fetch_option_price(ticker):
    try:
        url = f"https://api.nasdaq.com/api/quote/{ticker}/option-chain?assetclass=stocks&limit=3000&fromdate=all&todate=undefined&excode=oprac&callput=callput&money=all&type=all"

        headers = {
            'accept': 'application/json, text/plain, */*',
            'accept-language': 'en-US,en;q=0.9',
            'origin': 'https://www.nasdaq.com',
            'referer': 'https://www.nasdaq.com/',
            'sec-ch-ua': '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Linux"',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-site',
            'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
        }

        response = requests.get(url, headers=headers)
        data = response.json()
        rows = data['data']['table']['rows']

        filtered_rows = [row for row in rows if row['c_Openinterest'] is not None or row['p_Openinterest'] is not None]
        df = pd.DataFrame(filtered_rows)

        for index, row in df.iterrows():
            df.at[index, 'expiryDate'] = convert_expiry_date(row['expiryDate'])
            df.at[index, 'c_Bid'] = convert_to_float(row['c_Bid'])
            df.at[index, 'c_Ask'] = convert_to_float(row['c_Ask'])
            df.at[index, 'c_Last'] = convert_to_float(row['c_Last'])
            df.at[index, 'c_Change'] = convert_to_float(row['c_Change'])
            df.at[index, 'c_Openinterest'] = convert_to_int(row['c_Openinterest'])
            df.at[index, 'c_Volume'] = convert_to_int(row['c_Volume'])
            df.at[index, 'p_Bid'] = convert_to_float(row['p_Bid'])
            df.at[index, 'p_Ask'] = convert_to_float(row['p_Ask'])
            df.at[index, 'p_Last'] = convert_to_float(row['p_Last'])
            df.at[index, 'p_Change'] = convert_to_float(row['p_Change'])
            df.at[index, 'p_Openinterest'] = convert_to_int(row['p_Openinterest'])
            df.at[index, 'p_Volume'] = convert_to_int(row['p_Volume'])
            df.at[index, 'close'] = stock_prices['Adj Close'][ticker].iloc[-1]

        df['ticker'] = ticker
        df = df.drop(columns=['drillDownURL', 'c_colour', 'p_colour', 'expirygroup'])

        return df
    except (ValueError, TypeError):
        return None

ticker = "HRMY"
option_data = fetch_option_price(ticker)
print(option_data)

    expiryDate c_Last c_Change c_Bid c_Ask c_Volume c_Openinterest strike  \
0   2024-10-18   20.6     None  23.3  25.9     None            235  15.00   
1   2024-10-18   None     None  20.6  23.3     None           None  17.50   
2   2024-10-18   None     None  18.2  20.8     None           None  20.00   
3   2024-10-18   None     None  15.1  18.3     None           None  22.50   
4   2024-10-18   14.7     None  12.6  16.3     None              5  25.00   
5   2024-10-18   9.75     None   8.1  11.0     None              9  30.00   
6   2024-10-18   4.55     None   4.3   6.5     None             55  35.00   
7   2024-10-18    2.1    -0.55   1.2   3.2       22            269  40.00   
8   2024-10-18    0.7     None  0.25  1.25     None            145  45.00   
9   2024-10-18   0.25     None   0.2  1.75     None            253  50.00   
10  2024-11-15   None     None  17.6  21.3     None           None  20.00   
11  2024-11-15   None     None  15.6  18.8     None           None  22.50   

In [97]:
def price_heston_option(
    spot_price, 
    strike_price, 
    risk_free_rate, 
    dividend_yield, 
    initial_volatility, 
    expiry, 
    kappa, 
    theta, 
    sigma, 
    rho, 
    option_type='call'
):
    spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
    rate_handle = ql.YieldTermStructureHandle(
        ql.FlatForward(0, ql.NullCalendar(), ql.QuoteHandle(ql.SimpleQuote(risk_free_rate)), ql.Actual360())
    )
    dividend_handle = ql.YieldTermStructureHandle(
        ql.FlatForward(0, ql.NullCalendar(), ql.QuoteHandle(ql.SimpleQuote(dividend_yield)), ql.Actual360())
    )

    heston_process = ql.HestonProcess(
        rate_handle,  # Risk-free rate
        dividend_handle,  # Dividend yield
        spot_handle,  # Initial stock price
        initial_volatility ** 2,  # Initial variance (square of initial volatility)
        kappa,  # Mean reversion speed
        theta,  # Long-term variance
        sigma,  # Volatility of volatility
        rho  # Correlation
    )
    heston_model = ql.HestonModel(heston_process)

    expiry_date = ql.Date().todaysDate() + ql.Period(int(expiry), ql.Days)
    exercise = ql.EuropeanExercise(expiry_date)
    payoff_type = ql.Option.Call if option_type.lower() == 'call' else ql.Option.Put
    payoff = ql.PlainVanillaPayoff(payoff_type, strike_price)
    european_option = ql.VanillaOption(payoff, exercise)

    engine = ql.AnalyticHestonEngine(heston_model)
    european_option.setPricingEngine(engine)

    option_price = european_option.NPV()
    return option_price

In [98]:
def fetch_and_calculate_garch_volatility(ticker, start_date, end_date, p=1, q=1):
    """
    Fetch historical stock price data, calculate daily returns, and compute GARCH volatility.
    
    Parameters:
    - ticker: The stock ticker symbol (e.g., 'AAPL').
    - start_date: The start date for fetching data (format 'YYYY-MM-DD').
    - end_date: The end date for fetching data (format 'YYYY-MM-DD').
    - p: The order of the GARCH term.
    - q: The order of the ARCH term.
    
    Returns:
    - A pandas Series of estimated conditional volatilities.
    """
    
    stock_data = yf.download(ticker, start=start_date, end=end_date)

    returns = stock_data['Adj Close'].pct_change().dropna()

    model = arch_model(returns, vol='Garch', p=p, q=q, rescale=False)
    model_fit = model.fit(disp="off")
    daily_volatility = model_fit.conditional_volatility

    annual_volatility = daily_volatility * np.sqrt(252)

    return annual_volatility

In [99]:
def fetch_and_calculate_std(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)

    returns = stock_data['Adj Close'].pct_change().dropna()

    daily_volatility = returns.std()

    annual_volatility = daily_volatility * np.sqrt(252)

    return annual_volatility

In [100]:

def screen_option(ticker):
    try:
        garch_3m = fetch_and_calculate_garch_volatility(ticker, start_date_3m, end_date, p=1, q=1)
        std_3m = fetch_and_calculate_std(ticker, start_date_3m, end_date)
        std_12m = fetch_and_calculate_std(ticker, start_date_12m, end_date)
        vol_of_vol = garch_3m.std()
        option_prices = fetch_option_price(ticker)

        option_df = pd.DataFrame(option_prices)

        for index, row in option_df.iterrows():
            expiry_day = (pd.to_datetime(row['expiryDate']).tz_localize('UTC') - pd.to_datetime(end_date).tz_localize('UTC')).days

            c_price_theory = price_heston_option(
                spot_price=float(row['close']),
                strike_price=float(row['strike']),
                risk_free_rate=risk_free_rate,
                dividend_yield=0.0,
                initial_volatility=std_3m,
                expiry=expiry_day,
                kappa=1.0,
                theta=std_12m,
                sigma=vol_of_vol,
                rho=-0.5,
                option_type='call'
            )

            p_price_theory = price_heston_option(
                spot_price=float(row['close']),
                strike_price=float(row['strike']),
                risk_free_rate=risk_free_rate,
                dividend_yield=0.0,
                initial_volatility=std_3m,
                expiry=expiry_day,
                kappa=1.0,
                theta=std_12m,
                sigma=vol_of_vol,
                rho=-0.5,
                option_type='put'
            )

            option_df.at[index, 'c_PriceTheory'] = c_price_theory
            option_df.at[index, 'p_PriceTheory'] = p_price_theory
            option_df.at[index, 'garchVol'] = garch_3m.iloc[-1]
            option_df.at[index, 'std'] = std_3m



        return option_df
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        return ticker, pd.DataFrame()

def screen_all_options(tickers):
    all_options_df = pd.DataFrame()

    for ticker in tickers:
        print(ticker)
        options_df = screen_option(ticker)
        options_df['ticker'] = ticker
        if not options_df.empty:
            all_options_df = pd.concat([all_options_df, options_df], ignore_index=True)

    return all_options_df

options_data = screen_all_options(tickers)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

CLOV



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

ASTS



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

...

HRMY



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

WLFC



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

FTAI


In [101]:
for index, row in options_data.iterrows():
  c_Signal = None
  p_Signal = None
  c_Reward = 0
  p_Reward = 0
  print(row)
  
  try:
    if row['c_PriceTheory'] >= 0.1 and row['c_Bid'] >= 0.1 and \
      row['c_Bid'] != None and \
      row['c_PriceTheory'] * 1.5 <= row['c_Bid']:
      c_Signal = "SELL"
      c_Reward = (row['c_Bid'] - row['c_PriceTheory']) / row['c_PriceTheory']
    elif row['c_PriceTheory'] >= 0.1 and row['c_Ask'] >= 0.1 and \
      row ['c_Ask'] != None and \
      row['c_Ask'] * 1.5 <= row['c_PriceTheory']:
      c_Signal = "BUY"
      c_Reward = (row['c_PriceTheory'] - row['c_Ask']) / row['c_Ask']


    if row['p_PriceTheory'] >= 0.1 and row['p_Bid'] >= 0.1 and \
      row['p_Bid'] != None and \
      row['p_PriceTheory'] * 1.5 <= row['p_Bid']:
      print_Signal = "SELL"
      p_Reward = (row['p_Bid'] - row['p_PriceTheory']) / row['p_PriceTheory']
    elif row['p_PriceTheory'] >= 0.1 and row['p_Ask'] >= 0.1 and \
      row ['p_Ask'] != None and \
      row['p_Ask'] * 1.5 <= row['p_PriceTheory']:
      p_Signal = "BUY"
      p_Reward = (row['p_PriceTheory'] - row['p_Ask']) / row['p_Ask']
  except (ValueError, TypeError):
    None

  options_data.loc[index, 'c_Signal'] = c_Signal
  options_data.loc[index, 'c_Reward'] = c_Reward
  options_data.loc[index, 'p_Signal'] = p_Signal
  options_data.loc[index, 'p_Reward'] = p_Reward

summary_df = pd.DataFrame(options_data)
summary_df = summary_df.sort_values(by=['p_Reward', 'c_Reward'], ascending=[False, False])
summary_df.to_csv('options_chain.csv')

print(summary_df)

expiryDate        2024-10-04
c_Last                  2.34
c_Change                None
c_Bid                   2.19
c_Ask                   2.72
c_Volume                None
c_Openinterest             1
strike                  0.50
p_Last                  0.02
p_Change                None
p_Bid                   None
p_Ask                   0.02
p_Volume                None
p_Openinterest            67
close                   2.87
ticker                  CLOV
c_PriceTheory       2.370486
p_PriceTheory           -0.0
garchVol            0.724082
std                 1.000471
Name: 0, dtype: object
expiryDate        2024-10-04
c_Last                  1.99
c_Change                None
c_Bid                   1.38
c_Ask                   2.22
c_Volume                None
c_Openinterest             4
strike                  1.00
p_Last                  0.01
p_Change                None
p_Bid                   None
p_Ask                   0.01
p_Volume                None
p_Openinterest      